In [1]:
import pennylane as qml
import numpy as np

In [ ]:
## code for VGQEC 3 encoding
def VGQEC_3_encoding(alpha, wires):
    A = (1.0 + np.exp(1j * alpha)) / 2.0
    B = (1.0 - np.exp(1j * alpha)) / 2.0
    ancillas = np.array([A, 0, 0, B], dtype=complex)
    qml.MottonenStatePreparation(ancillas, wires=[1, 2])

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])




In [ ]:
#testing the encoding circuit
dev = qml.device("default.qubit", wires=3)
@qml.qnode(dev)
def VGQEC_3(alpha, wires):
    # qml.PauliX(wires=0) # Initial state 
    VGQEC_3_encoding(alpha, wires)

    # return qml.state()

# print(VGQEC_3(np.pi, dev.wires))

[-1.34442674e-33+6.12323400e-17j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j  1.00000000e+00+2.58605321e-17j
  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j
  0.00000000e+00+0.00000000e+00j  0.00000000e+00+0.00000000e+00j]


In [25]:
#variational recovery circuit for 3 qubits
def recovery_variational_3(beta, L , wires):
    for i in range(2):
        qml.RZ(beta[1], wires=i)

    for l in range(L):
        for i in range(2):
            qml.RX(beta[0], wires=i)
            qml.RZ(beta[1], wires=i)        
    
        qml.CZ(beta[2], wires=[0, 1])
        qml.CZ(beta[2], wires=[1, 2])
        qml.CZ(beta[2], wires=[0, 2])
        

    for i in range(wires):
        qml.RX(beta[0], wires=i)
        qml.RZ(beta[1], wires=i)

In [ ]:
#reovery map circuit for 3 qubits (from the repetition code)
def repetition_code_recovery_3(alpha, wires):
    for wire in wires:
        qml.Hadamard(wires=wire)
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[0, 1])
    qml.Toffoli(wires=[2, 1, 0])


In [112]:
## code for VGQEC 3 encoding
def VGQEC_3_encoding(alpha, wires):
    A = (1.0 + np.exp(1j * alpha)) / 2.0
    B = (1.0 - np.exp(1j * alpha)) / 2.0
    ancillas = np.array([A, 0, 0, B], dtype=complex)
    qml.MottonenStatePreparation(ancillas, wires=[1, 2])

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])


#variational recovery circuit for 3 qubits
def recovery_variational_3(beta, L , wires):
    for i in range(2):
        qml.RZ(beta[1], wires=i)

    for l in range(L):
        for i in range(2):
            qml.RX(beta[0], wires=i)
            qml.RZ(beta[1], wires=i)

        qml.CZ(wires=[0, 1])
        qml.CZ(wires=[1, 2])
        qml.CZ(wires=[0, 2])


    for i in range(2):
        qml.RX(beta[0], wires=i)
        qml.RZ(beta[1], wires=i)

#reovery map circuit for 3 qubits (from the repetition code)
def repetition_code_recovery_3(alpha, wires):
# 1. Undo the two CNOTs
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[0, 1])
    
    # 2. Undo ancilla preparation
    #    We apply the inverse of the state prep:
    A = (1.0 + np.exp(1j * alpha)) / 2.0
    B = (1.0 - np.exp(1j * alpha)) / 2.0
    ancillas_dagger = np.array([np.conj(A), 0, 0, np.conj(B)], dtype=complex)
    qml.MottonenStatePreparation(ancillas_dagger, wires=[1, 2])


#test for VGQEC code with 3 qubits
dev3_VGQEC_circuit = qml.device("default.mixed", wires=3)
@qml.qnode(dev3_VGQEC_circuit)
def circuit_3(wires, alpha, beta):
    qml.Hadamard(wires=0)
    #encoding
    VGQEC_3_encoding(alpha, wires)
    #noise
    qml.AmplitudeDamping(beta, wires=[0])
    # qml.PauliZ(wires=0)
    #recovery
    # recovery_variational_3([np.pi/2, np.pi/2, np.pi/2], 2, wires)
    repetition_code_recovery_3(alpha, wires)

    return qml.density_matrix(wires=0)

print(circuit_3(dev3_VGQEC_circuit.wires, np.pi/2, 0.9))

[[0.95      +0.00000000e+00j 0.15811388+0.00000000e+00j]
 [0.15811388+0.00000000e+00j 0.05      +2.60208521e-18j]]


In [98]:
print(circuit_3(dev3_VGQEC_circuit.wires, 0.1))

[[ 5.00000000e-01+1.38777878e-17j  1.11022302e-16+0.00000000e+00j]
 [-1.11022302e-16+1.38777878e-17j  5.00000000e-01+2.77555756e-17j]]
